In [ ]:
# =========================================================
# MULTIPLEX — MEMORY vs AMNESIC (corr MAX) — OPTIMISÉ
# - Graphes FIXES par (topo,z,corr)
# - Influence layers précomputées (BFS cutoff Lmax)
# - Numba: AUCUNE allocation dans la boucle
# - Même seeds IC pour comparer amnesic vs memory
# =========================================================

import numpy as np
import networkx as nx
import pandas as pd
import os, time
from collections import deque
from numba import njit
from joblib import Parallel, delayed

# ------------------------
# GRAPH BUILDERS
# ------------------------
def make_graph(topo, N, z, seed):
    if topo == "BA":
        m = int(z / 2)
        return nx.barabasi_albert_graph(N, m, seed=seed)
    elif topo == "ER":
        return nx.erdos_renyi_graph(N, z / (N - 1), seed=seed)
    else:
        raise ValueError("topo must be 'BA' or 'ER'")

def make_influence_graph(G_g, corr, N, seed_perm=777):
    """corr=max => G_i = G_g ; corr=nulle => permutation FIXE."""
    if corr == "max":
        return G_g
    rng = np.random.default_rng(seed_perm)
    perm = np.arange(N, dtype=np.int32)
    rng.shuffle(perm)
    mapping = {int(i): int(perm[i]) for i in range(N)}
    return nx.relabel_nodes(G_g, mapping)

def graph_to_csr(G):
    """CSR-like adjacency for fast numba iteration."""
    G = nx.convert_node_labels_to_integers(G)
    N = G.number_of_nodes()
    adj_f = []
    adj_o = [0]
    for i in range(N):
        neigh = list(G.neighbors(i))
        adj_f.extend(neigh)
        adj_o.append(len(adj_f))
    adj_f = np.array(adj_f, dtype=np.int32)
    adj_o = np.array(adj_o, dtype=np.int32)
    deg = (adj_o[1:] - adj_o[:-1]).astype(np.int32)
    return adj_f, adj_o, deg

def build_influence_layers_bfs(G_i, N, Lmax):
    """
    Precompute influence neighbors up to distance Lmax.
    Returns:
      inf_indices: (Lmax, N, N) int32 (ok for N=500)
      inf_counts : (Lmax, N) int32
    """
    adj = [list(G_i.neighbors(i)) for i in range(N)]
    inf_indices = np.zeros((Lmax, N, N), dtype=np.int32)
    inf_counts  = np.zeros((Lmax, N), dtype=np.int32)

    for src in range(N):
        dist = np.full(N, -1, dtype=np.int32)
        dist[src] = 0
        q = deque([src])

        while q:
            u = q.popleft()
            du = dist[u]
            if du == Lmax:
                continue
            for v in adj[u]:
                if dist[v] == -1:
                    dist[v] = du + 1
                    q.append(v)

        for j in range(N):
            d = dist[j]
            if 0 < d <= Lmax:
                l = d - 1
                c = inf_counts[l, src]
                inf_indices[l, src, c] = j
                inf_counts[l, src] += 1

    return inf_indices, inf_counts

# ------------------------
# NUMBA CORE (NO ALLOCS IN LOOP)
# ------------------------
@njit(cache=True, fastmath=True)
def run_simulation_memory_vs_amnesic_fast(
    adj_f, adj_o, deg_f,
    inf_indices, inf_counts,
    alpha_weights,
    theta, b,
    steps,
    vigilance_mode,  # 0 amnesic / 1 memory(sticky)
    cost_c,
    seed
):
    np.random.seed(seed)

    N = deg_f.shape[0]
    L = alpha_weights.shape[0]

    strat = np.random.randint(0, 2, N).astype(np.float64)  # 1=C, 0=D

    vig = np.zeros(N, dtype=np.float64)
    for i in range(N):
        if strat[i] == 1.0 and np.random.random() < 0.5:
            vig[i] = 1.0

    # prealloc
    influences = np.empty(N, dtype=np.float64)
    T_eff      = np.empty(N, dtype=np.float64)
    payoffs    = np.empty(N, dtype=np.float64)
    new_strat  = np.empty(N, dtype=np.float64)

    # stop criterion (rolling std)
    history_window = 300
    burn_in = 1200
    tol_std = 8e-4
    history_c = np.zeros(history_window, dtype=np.float64)
    filled = 0

    for t in range(steps):

        # A) Influence
        for i in range(N):
            I_i = 0.0
            for l in range(L):
                cnt = inf_counts[l, i]
                if cnt > 0:
                    s = 0.0
                    for k in range(cnt):
                        s += vig[inf_indices[l, i, k]]
                    I_i += alpha_weights[l] * (s / cnt)
            influences[i] = I_i

        # B) Vigilance update
        if vigilance_mode == 0:  # amnesic
            for i in range(N):
                if strat[i] == 1.0:
                    vig[i] = 1.0 if influences[i] >= theta else 0.0
                else:
                    vig[i] = 0.0
        else:  # sticky memory
            for i in range(N):
                if strat[i] == 0.0:
                    vig[i] = 0.0
                else:
                    if vig[i] == 1.0:
                        vig[i] = 1.0
                    else:
                        vig[i] = 1.0 if influences[i] >= theta else 0.0

        # C) Payoffs
        for i in range(N):
            T_eff[i] = 1.0 + (b - 1.0) * (1.0 - influences[i])
            payoffs[i] = 0.0

        for i in range(N):
            start = adj_o[i]
            end = adj_o[i + 1]

            if strat[i] == 1.0:
                for idx in range(start, end):
                    j = adj_f[idx]
                    if strat[j] == 1.0:
                        payoffs[i] += 1.0
            else:
                Ti = T_eff[i]
                for idx in range(start, end):
                    j = adj_f[idx]
                    if strat[j] == 1.0:
                        payoffs[i] += Ti

        if cost_c > 0.0:
            for i in range(N):
                if vig[i] == 1.0:
                    payoffs[i] -= cost_c

        # D) Strategy update (PROPORTIONAL only, like your config)
        for i in range(N):
            new_strat[i] = strat[i]

        for i in range(N):
            start = adj_o[i]
            end = adj_o[i + 1]
            k_i = end - start
            if k_i == 0:
                continue

            j = adj_f[start + np.random.randint(0, k_i)]
            if payoffs[j] > payoffs[i]:
                k_j = deg_f[j]
                kmax = k_i if k_i > k_j else k_j
                Te = T_eff[i] if T_eff[i] > T_eff[j] else T_eff[j]
                phi = kmax * Te
                if phi > 0.0:
                    if np.random.random() < (payoffs[j] - payoffs[i]) / phi:
                        new_strat[i] = strat[j]

        for i in range(N):
            strat[i] = new_strat[i]

        # rho
        rho = 0.0
        for i in range(N):
            rho += strat[i]
        rho /= N

        if rho <= 0.0 or rho >= 1.0:
            mv = 0.0
            for i in range(N):
                mv += vig[i]
            return rho, mv / N

        history_c[t % history_window] = rho
        if filled < history_window:
            filled += 1

        if t > burn_in and (t % 50 == 0) and (filled == history_window):
            m = 0.0
            for kk in range(history_window):
                m += history_c[kk]
            m /= history_window

            v = 0.0
            for kk in range(history_window):
                d = history_c[kk] - m
                v += d * d
            v /= history_window

            if np.sqrt(v) < tol_std:
                mv = 0.0
                for i in range(N):
                    mv += vig[i]
                return m, mv / N

    # fallback
    m = 0.0
    for kk in range(filled if filled > 0 else 1):
        m += history_c[kk]
    m /= (filled if filled > 0 else 1)

    mv = 0.0
    for i in range(N):
        mv += vig[i]
    return m, mv / N

# ------------------------
# CONFIG (same spirit as yours)
# ------------------------
N = 500
REPS = 100
steps = 12000

corr = "max"
cost_c = 0.0
B_RANGE = np.around(np.arange(1.0, 2.1, 0.1), 1)

TOPOS = [("BA", 16)]
thetas = [0.4]
L_list = [4]

OUTFILE = "P2_2_MEMORY_vs_AMNESIC_corrMAX_cost0_prop_OPT.csv"
OUTFILE = "P2_2_BA16_VÉRIFICATION.csv"

# seeds IC shared between modes
BASE_SEED = 12345
IC_SEEDS = np.array([BASE_SEED + 1000*r for r in range(REPS)], dtype=np.int64)

def one_ic(seed, adj_f, adj_o, deg_f, inf_idx_L, inf_cnt_L, alphas, theta, b, steps, mode, cost_c):
    return run_simulation_memory_vs_amnesic_fast(
        adj_f, adj_o, deg_f,
        inf_idx_L, inf_cnt_L,
        alphas,
        float(theta), float(b),
        int(steps),
        int(mode),
        float(cost_c),
        int(seed)
    )

# ------------------------
# Warmup compile
# ------------------------
def warmup_compile():
    Gtmp = make_graph("ER", 60, 8, seed=0)
    Gi = make_influence_graph(Gtmp, "max", 60, seed_perm=1)
    af, ao, deg = graph_to_csr(Gtmp)
    inf_idx, inf_cnt = build_influence_layers_bfs(Gi, 60, Lmax=2)
    alphas = np.array([1.0], dtype=np.float64)

    _ = run_simulation_memory_vs_amnesic_fast(
        af, ao, deg,
        inf_idx[:1], inf_cnt[:1],
        alphas,
        0.6, 1.4,
        200,
        0,
        0.0,
        0
    )

warmup_compile()

# ------------------------
# RUN
# ------------------------
for topo, z in TOPOS:
    # FIXED graphs for this topo/z
    Gg = make_graph(topo, N, z, seed=42)
    Gi = make_influence_graph(Gg, corr=corr, N=N, seed_perm=777)

    # precompute game CSR once
    adj_f, adj_o, deg_f = graph_to_csr(Gg)

    # precompute influence once up to max L
    Lmax = max(L_list)
    inf_idx_all, inf_cnt_all = build_influence_layers_bfs(Gi, N, Lmax=Lmax)

    for theta in thetas:
        for L in L_list:
            inf_idx_L = inf_idx_all[:L]
            inf_cnt_L = inf_cnt_all[:L]

            alphas = np.array([0.5**l for l in range(L)], dtype=np.float64)
            alphas /= np.sum(alphas)

            for mode_name, mode_val in [("amnesic", 0), ("memory", 1)]:
                for b in B_RANGE:
                    start = time.time()

                    data = Parallel(n_jobs=-1)(
                        delayed(one_ic)(
                            int(IC_SEEDS[r]),
                            adj_f, adj_o, deg_f,
                            inf_idx_L, inf_cnt_L,
                            alphas, theta, float(b), steps,
                            mode_val, cost_c
                        )
                        for r in range(REPS)
                    )

                    rhos = np.array([x[0] for x in data], dtype=np.float64)
                    vigs = np.array([x[1] for x in data], dtype=np.float64)

                    pd.DataFrame([{
                        "Topo": topo, "Z": z, "Corr": corr,
                        "Theta": float(theta), "L": int(L), "b": float(b),
                        "Update": "proportional", "K": np.nan,
                        "VigilanceMode": mode_name, "Cost": float(cost_c),
                        "C_mean": float(rhos.mean()), "C_std": float(rhos.std()),
                        "V_mean": float(vigs.mean()), "V_std": float(vigs.std())
                    }]).to_csv(OUTFILE, mode="a",
                               header=not os.path.exists(OUTFILE), index=False)

                    print(f"[{topo} z={z}] {mode_name} Th={theta} L={L} b={b} | {time.time()-start:.1f}s")
